# 18 - HBase jako Serving Layer dla rekomendacji

Wykorzystanie HBase jako warstwy serwującej rekomendacje w systemie produkcyjnym.

**Tematy:**
- Architektura serving layer z HBase
- Row key design: user_id + CF recs + rank columns
- ALS training i eksport rekomendacji do HBase
- Bulk load vs individual puts
- TTL i automatyczne wygasanie rekomendacji
- Wersjonowanie rekomendacji z HBase timestamps
- Filtrowanie z HBase Scan filters
- Tabela podobnych filmów w HBase
- Pełny pipeline: train, export, serve

## 1. Architektura Serving Layer

W systemie rekomendacji rozdzielamy **offline training** od **online serving**:

```
┌─────────────────────────────────────────────────────────────────┐
│                    OFFLINE PIPELINE (batch)                     │
│                                                                 │
│  PostgreSQL ──► Spark ──► ALS Training ──► Top-K Recs          │
│  (ratings)      (ETL)     (MLlib)          (per user)          │
│                                                │                │
│                                    Bulk Load   │                │
│                                                ▼                │
│                                    ┌───────────────────┐       │
│                                    │      HBase        │       │
│                                    │                   │       │
│                                    │ user_recs table   │       │
│                                    │ movie_sim table   │       │
│                                    └─────────┬─────────┘       │
└──────────────────────────────────────────────┼─────────────────┘
                                               │
┌──────────────────────────────────────────────┼─────────────────┐
│                    ONLINE SERVING (real-time)  │                │
│                                               │                │
│  User Request ──► FastAPI ──► HBase GET ──────┘                │
│  (GET /recs/42)   (API)      (< 10ms!)                        │
│                      │                                         │
│                      ▼                                         │
│              JSON Response                                     │
│              {recs: [...]}                                     │
└────────────────────────────────────────────────────────────────┘
```

### Dlaczego HBase jako serving layer?

| Cecha | HBase | PostgreSQL | Redis |
|-------|-------|------------|-------|
| Latencja GET | ~1-5ms | ~5-20ms | ~0.5ms |
| Skalowalność | Horyzontalna | Wertykalna | Ograniczona RAM |
| Dane na dysku | Tak (HDFS) | Tak | Opcjonalnie |
| TTL per cell | Tak | Nie (trigger) | Tak |
| Wersjonowanie | Wbudowane | Ręczne | Nie |
| Koszt 1TB | Niski (HDFS) | Średni | Wysoki (RAM!) |

## 2. Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
import json
import time

spark = SparkSession.builder \
    .appName("18_HBase_Serving_Layer") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages",
            "org.postgresql:postgresql:42.7.1,"
            "org.apache.hbase.connectors.spark:hbase-spark:1.0.1,"
            "org.apache.hbase:hbase-client:2.6.1,"
            "org.apache.hbase:hbase-common:2.6.1,"
            "org.apache.hbase:hbase-mapreduce:2.6.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.hadoop.hbase.zookeeper.quorum", "hbase-zookeeper") \
    .config("spark.hadoop.hbase.zookeeper.property.clientPort", "2181") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
jdbc_props = {"user": "recommender", "password": "recommender", "driver": "org.postgresql.Driver"}

print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

In [ ]:
# Zaladuj dane
ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=jdbc_props,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=jdbc_props)

ratings.cache()
movies.cache()
print(f"Ratings: {ratings.count()}, Movies: {movies.count()}")

## 3. Row Key Design dla rekomendacji

```
Tabela: user_recommendations
┌──────────────┬────────────────────────────────────────────────────────┐
│ Row Key      │ Column Family: recs                                   │
│ (user_id)    ├───────────┬───────────┬───────────┬─────┬────────────┤
│              │ recs:r_01 │ recs:r_02 │ recs:r_03 │ ... │ recs:r_20  │
├──────────────┼───────────┼───────────┼───────────┼─────┼────────────┤
│ 000042       │ 318       │ 858       │ 527       │ ... │ 4993       │
│              │ @t2 3.95  │ @t2 3.89  │ @t2 3.85  │     │ @t2 3.21   │
│              │ @t1 296   │ @t1 1196  │ @t1 260   │     │ @t1 2571   │
├──────────────┼───────────┼───────────┼───────────┼─────┼────────────┤
│ 000100       │ 2571      │ 1        │ 4993      │ ... │ 110        │
└──────────────┴───────────┴───────────┴───────────┴─────┴────────────┘

Kazdy rank column przechowuje movie_id.
@t2 = nowsza wersja rekomendacji (po retrenowaniu modelu)
@t1 = starsza wersja (automatycznie dostepna dzieki wersjonowaniu HBase)
```

### Zalety tego designu:
- **Jeden GET** pobiera wszystkie 20 rekomendacji dla usera
- **Wersjonowanie** daje historię zmian rekomendacji
- **TTL** automatycznie usuwa przestarzałe rekomendacje
- **Row key = user_id** - naturalny access pattern API

## 4. Trening ALS i generowanie rekomendacji

In [ ]:
# Podział danych
(training, test) = ratings.randomSplit([0.8, 0.2], seed=42)
training.cache()

# Trening modelu ALS
als = ALS(
    maxIter=10,
    regParam=0.1,
    rank=20,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop",
    seed=42
)

start = time.time()
model = als.fit(training)
train_time = time.time() - start
print(f"Model ALS wytrenowany w {train_time:.1f}s (rank={model.rank})")

# Ewaluacja
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"RMSE na zbiorze testowym: {rmse:.4f}")

In [ ]:
# Generowanie top 20 rekomendacji dla WSZYSTKICH uzytkownikow
TOP_K = 20

start = time.time()
all_recs = model.recommendForAllUsers(TOP_K)
all_recs.cache()
num_users = all_recs.count()
recs_time = time.time() - start

print(f"Wygenerowano rekomendacje dla {num_users} uzytkownikow w {recs_time:.1f}s")
all_recs.show(3, truncate=False)

In [ ]:
# Rozpakuj rekomendacje do formatu plaskiego (flat)
# Z: user_id, [{movie_id, rating}, ...]
# Do: user_id, r_01, r_02, ..., r_20 (movie_ids), s_01, s_02, ..., s_20 (scores)

recs_flat = all_recs.select(
    col("user_id"),
    *[col("recommendations")[i]["movie_id"].alias(f"r_{i+1:02d}") for i in range(TOP_K)],
    *[round(col("recommendations")[i]["rating"], 4).alias(f"s_{i+1:02d}") for i in range(TOP_K)]
)

# Dodaj row key (zero-padded user_id)
recs_flat = recs_flat.withColumn(
    "user_key", lpad(col("user_id").cast("string"), 6, "0")
)

print(f"Kolumny: {len(recs_flat.columns)}")
recs_flat.select("user_key", "r_01", "r_02", "r_03", "s_01", "s_02", "s_03").show(5)

## 5. Eksport rekomendacji do HBase

In [ ]:
# Definicja katalogu HBase dla rekomendacji
recs_columns = {
    "user_key": {"cf": "rowkey", "col": "user_key", "type": "string"},
    "user_id":  {"cf": "meta", "col": "user_id", "type": "int"}
}

# Dodaj kolumny r_01..r_20 (movie_ids) do CF "recs"
for i in range(1, TOP_K + 1):
    recs_columns[f"r_{i:02d}"] = {"cf": "recs", "col": f"r_{i:02d}", "type": "int"}

# Dodaj kolumny s_01..s_20 (scores) do CF "scores"
for i in range(1, TOP_K + 1):
    recs_columns[f"s_{i:02d}"] = {"cf": "scores", "col": f"s_{i:02d}", "type": "float"}

recs_catalog = json.dumps({
    "table": {"namespace": "default", "name": "user_recommendations"},
    "rowkey": "user_key",
    "columns": recs_columns
})

print(f"Katalog ma {len(recs_columns)} kolumn")
print(json.dumps(json.loads(recs_catalog), indent=2)[:500] + "\n...")

In [ ]:
# Zapis rekomendacji do HBase
start = time.time()

recs_flat.write \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=recs_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .option("newTable", "5") \
    .mode("overwrite") \
    .save()

write_time = time.time() - start
print(f"Rekomendacje zapisane do HBase w {write_time:.1f}s ({num_users} uzytkownikow)")

In [ ]:
# Weryfikacja - odczytaj rekomendacje z HBase
recs_from_hbase = spark.read \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=recs_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .load()

print(f"Rekomendacje w HBase: {recs_from_hbase.count()} uzytkownikow")

# Pokaz rekomendacje dla user 42
user_42 = recs_from_hbase.filter(col("user_key") == "000042")
user_42.show(1, truncate=False)

In [ ]:
# Pokaz rekomendacje z tytulami filmow
user_42_recs = user_42.select(
    *[col(f"r_{i:02d}").alias("movie_id") for i in range(1, 6)]
)

# Rozpakuj do wierszy
rec_rows = []
u42 = user_42.collect()[0]
for i in range(1, TOP_K + 1):
    movie_id = u42[f"r_{i:02d}"]
    score = u42[f"s_{i:02d}"]
    rec_rows.append((i, movie_id, float(score)))

rec_df = spark.createDataFrame(rec_rows, ["rank", "movie_id", "predicted_score"])
rec_df.join(movies, "movie_id") \
    .select("rank", "title", "genres", "predicted_score") \
    .orderBy("rank") \
    .show(TOP_K, truncate=False)

## 6. Bulk Load vs Individual Puts - porownanie wydajnosci

Dwa sposoby zapisu danych do HBase:
- **Individual Puts**: kazdy wiersz osobno przez RegionServer (wolne)
- **Bulk Load**: generuj HFiles bezposrednio i zaladuj do RegionServer (szybkie)

```
Individual Puts:                    Bulk Load:
                                    
Client ──► RegionServer             Spark ──► HFiles (HDFS)
  │              │                               │
  │  WAL write   │                  HBase ◄──── Load HFiles
  │  MemStore    │                  (completebulkload)
  │  flush       │                               
  ▼              ▼                  Pomija WAL i MemStore!
  HFile          HFile              Bezposrednio HFiles.
```

In [ ]:
# Benchmark: Individual Puts (standardowy zapis przez SHC)
sample_10k = recs_flat.limit(10000).cache()
sample_10k.count()  # force cache

# Metoda 1: Standardowy zapis (individual puts pod spodem)
puts_catalog = json.dumps({
    "table": {"namespace": "default", "name": "recs_puts_test"},
    "rowkey": "user_key",
    "columns": recs_columns
})

start = time.time()
sample_10k.write \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=puts_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .option("newTable", "5") \
    .mode("overwrite") \
    .save()
puts_time = time.time() - start
print(f"Individual Puts (10k rows): {puts_time:.1f}s")

In [ ]:
# Metoda 2: Bulk Load - generuj HFiles i zaladuj
# Wymaga posortowania po row key i zapisu jako HFile

bulk_catalog = json.dumps({
    "table": {"namespace": "default", "name": "recs_bulk_test"},
    "rowkey": "user_key",
    "columns": recs_columns
})

start = time.time()

# Bulk load wymaga posortowania po row key
sample_sorted = sample_10k.orderBy("user_key")

sample_sorted.write \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=bulk_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .option("newTable", "5") \
    .option("hbase.spark.bulkload.enable", "true") \
    .mode("overwrite") \
    .save()

bulk_time = time.time() - start
print(f"Bulk Load (10k rows): {bulk_time:.1f}s")
print(f"\nSpeedup: {puts_time / bulk_time:.1f}x szybciej z Bulk Load")

### Kiedy uzywac Bulk Load?

| Scenariusz | Metoda | Dlaczego |
|------------|--------|----------|
| Inicjalny zaladowanie danych | Bulk Load | Miliony wierszy, jednorazowo |
| Batch update rekomendacji | Bulk Load | Duza ilosc, co kilka godzin |
| Pojedyncze oceny uzytkownikow | Individual Put | Male dane, real-time |
| Streaming updates | Individual Put | Niski latency wymagany |

## 7. TTL - automatyczne wygasanie rekomendacji

HBase umozliwia ustawienie **TTL (Time To Live)** na column family. Po uplywie TTL dane sa automatycznie usuwane podczas compaction.

Przydatne dla rekomendacji - stare rekomendacje tracą aktualnosc!

In [ ]:
%%bash
# Tworzenie tabeli z TTL w HBase Shell
# TTL = 86400 (24 godziny) - rekomendacje wygasaja po 1 dniu
# TTL = 604800 (7 dni) - rekomendacje wygasaja po tygodniu

# hbase shell <<EOF
# create 'user_recommendations_ttl',
#   {NAME => 'recs', VERSIONS => 3, TTL => 604800, COMPRESSION => 'SNAPPY'},
#   {NAME => 'scores', VERSIONS => 3, TTL => 604800, COMPRESSION => 'SNAPPY'},
#   {NAME => 'meta', VERSIONS => 1, TTL => 2592000}
# EOF

# Sprawdz TTL:
# hbase shell -n <<< "describe 'user_recommendations_ttl'"

# Zmiana TTL na istniejacej tabeli:
# hbase shell <<EOF
# disable 'user_recommendations_ttl'
# alter 'user_recommendations_ttl', {NAME => 'recs', TTL => 86400}
# enable 'user_recommendations_ttl'
# EOF

echo "TTL configuration reference (uncomment to run)"

In [ ]:
# Symulacja TTL - zapisz rekomendacje z roznym timestamp
# W produkcji HBase automatycznie przypisuje timestamp

# Dodaj kolumne z timestamp generowania rekomendacji
recs_with_ts = recs_flat.limit(100).withColumn(
    "generated_at", current_timestamp().cast("string")
).withColumn(
    "expires_at",
    date_add(current_timestamp(), 7).cast("string")  # wygasa za 7 dni
)

print("Rekomendacje z timestamp i expiry:")
recs_with_ts.select("user_key", "generated_at", "expires_at", "r_01", "r_02").show(5)

## 8. Wersjonowanie - historia rekomendacji

HBase przechowuje wiele wersji (timestamps) tej samej komorki. Mozemy uzyc tego do sledzenia jak rekomendacje zmienialy sie w czasie.

```
Row: 000042
  recs:r_01
    @t=1700000000  movie_id=318   (Shawshank Redemption)
    @t=1699000000  movie_id=296   (Pulp Fiction)
    @t=1698000000  movie_id=2571  (Matrix)
  recs:r_02  
    @t=1700000000  movie_id=858   (Godfather)
    @t=1699000000  movie_id=318   (Shawshank Redemption)
    ...
    
GET z VERSIONS => 3  →  zwraca 3 ostatnie wersje rekomendacji
```

In [ ]:
%%bash
# Odczyt wersji w HBase Shell

# Odczyt najnowszej wersji (domyslnie):
# hbase shell -n <<< "get 'user_recommendations', '000042', {COLUMN => 'recs:r_01'}"

# Odczyt 3 ostatnich wersji:
# hbase shell -n <<< "get 'user_recommendations', '000042', {COLUMN => 'recs:r_01', VERSIONS => 3}"

# Odczyt wersji z zakresu czasowego:
# hbase shell <<EOF
# get 'user_recommendations', '000042', {COLUMN => 'recs:r_01', TIMERANGE => [1699000000000, 1700000000000]}
# EOF

echo "Versioning commands reference (uncomment to run)"

In [ ]:
# Symulacja wersjonowania - dwa rozne modele ALS
# Model 1: rank=10, regParam=0.1
# Model 2: rank=50, regParam=0.05

# Trenuj drugi model z innymi parametrami
als_v2 = ALS(
    maxIter=15,
    regParam=0.05,
    rank=50,
    userCol="user_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop",
    seed=123
)

model_v2 = als_v2.fit(training)
recs_v2 = model_v2.recommendForAllUsers(TOP_K)

# Porownaj rekomendacje v1 vs v2 dla user 42
recs_v1_42 = all_recs.filter(col("user_id") == 42) \
    .select(explode("recommendations").alias("rec")) \
    .select(col("rec.movie_id").alias("movie_id_v1"))

recs_v2_42 = recs_v2.filter(col("user_id") == 42) \
    .select(explode("recommendations").alias("rec")) \
    .select(col("rec.movie_id").alias("movie_id_v2"))

# Pokaz roznice
v1_ids = set(r.movie_id_v1 for r in recs_v1_42.collect())
v2_ids = set(r.movie_id_v2 for r in recs_v2_42.collect())
overlap = v1_ids & v2_ids

print(f"Model v1 (rank=20): {sorted(v1_ids)}")
print(f"Model v2 (rank=50): {sorted(v2_ids)}")
print(f"Wspolne filmy: {len(overlap)} / {TOP_K} ({len(overlap)/TOP_K*100:.0f}%)")
print(f"Nowe w v2: {v2_ids - v1_ids}")

W produkcji kazdy zapis do tej samej komorki automatycznie tworzy nowa wersje z aktualnym timestampem. Poprzednie wersje sa dostepne az do osiagniecia limitu VERSIONS lub TTL.

## 9. Scan z filtrami - segmentacja uzytkownikow

HBase oferuje rozne filtry do przeszukiwania danych. Przydatne do analizy rekomendacji per segment.

In [ ]:
%%bash
# HBase Scan z filtrami

# PrefixFilter - rekomendacje dla userow 000100-000199
# hbase shell <<EOF
# scan 'user_recommendations', {
#   STARTROW => '000100',
#   STOPROW => '000200',
#   COLUMNS => ['recs:r_01', 'recs:r_02', 'recs:r_03'],
#   LIMIT => 10
# }
# EOF

# SingleColumnValueFilter - userzy ktorym polecono film 318
# hbase shell <<EOF
# scan 'user_recommendations', {
#   FILTER => "SingleColumnValueFilter('recs', 'r_01', =, 'binary:318')",
#   LIMIT => 20
# }
# EOF

# ColumnPrefixFilter - tylko kolumny rekomendacji (bez scores)
# hbase shell <<EOF
# scan 'user_recommendations', {
#   FILTER => "ColumnPrefixFilter('r_')",
#   LIMIT => 5
# }
# EOF

# MultipleColumnPrefixFilter - rekomendacje i meta
# hbase shell <<EOF
# scan 'user_recommendations', {
#   FILTER => "MultipleColumnPrefixFilter('r_', 's_')",
#   STARTROW => '000042',
#   STOPROW => '000043'
# }
# EOF

echo "HBase filter commands reference (uncomment to run)"

In [ ]:
# Segmentacja przez Spark - "power users" z duza liczba ocen
user_activity = ratings.groupBy("user_id").agg(
    count("*").alias("num_ratings"),
    avg("rating").alias("avg_rating")
)

# Power users: > 500 ocen
power_users = user_activity.filter(col("num_ratings") > 500)
print(f"Power users (>500 ocen): {power_users.count()}")

# Polacz z rekomendacjami z HBase
power_user_recs = recs_from_hbase.join(
    power_users.withColumn("user_key", lpad(col("user_id").cast("string"), 6, "0")),
    "user_key"
)

print(f"Rekomendacje dla power users: {power_user_recs.count()}")
power_user_recs.select("user_key", "num_ratings", "avg_rating", "r_01", "r_02", "r_03").show(10)

## 10. Tabela podobnych filmow w HBase

Oprócz rekomendacji per uzytkownik, czesto potrzebujemy tez **podobnych filmow** (item-item similarity). HBase jest idealny do tego - szybki GET po movie_id.

```
Tabela: movie_similarity
┌──────────┬─────────────────────────────────────────────────┐
│ Row Key  │ Column Family: similar                          │
│(movie_id)├──────────┬──────────┬──────────┬──────┬────────┤
│          │sim:m_01  │sim:m_02  │sim:m_03  │ ...  │sim:m_10│
├──────────┼──────────┼──────────┼──────────┼──────┼────────┤
│ 000001   │ 3114     │ 2355     │ 588      │ ...  │ 1265   │
│(Toy Story)│(Toy St2) │(Bug Life)│(Aladdin) │      │(G Day) │
├──────────┼──────────┼──────────┼──────────┼──────┼────────┤
│ 000318   │ 858      │ 527      │ 1193     │ ...  │ 4226   │
│(Shawsh.) │(Godfth.) │(Schindl.)│(Matrix)  │      │(Memento)│
└──────────┴──────────┴──────────┴──────────┴──────┴────────┘
```

In [ ]:
# Oblicz item-item similarity z wektorow latentnych ALS
import numpy as np
from pyspark.sql.types import DoubleType

item_factors = model.itemFactors
print(f"Item factors: {item_factors.count()} filmow x rank={model.rank}")

# Dla kazdego filmu znajdz 10 najbardzej podobnych
# Uzyj cross-join + cosine similarity
# UWAGA: dla duzej liczby filmow to jest kosztowne - w produkcji uzyj LSH

# Weź top 1000 najpopularniejszych filmow (zeby nie eksplodowac)
popular_movies = ratings.groupBy("movie_id").count() \
    .orderBy(desc("count")).limit(1000) \
    .select("movie_id")

popular_factors = item_factors.join(
    popular_movies, item_factors["id"] == popular_movies["movie_id"]
).select(col("id").alias("movie_id"), "features")

print(f"Popularne filmy z wektorami: {popular_factors.count()}")

In [ ]:
# Cosine similarity miedzy wszystkimi parami (top 1000 filmow)
@udf(DoubleType())
def cosine_sim(v1, v2):
    a = np.array(v1)
    b = np.array(v2)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-8))

# Self-join dla par filmow
pairs = popular_factors.alias("a").crossJoin(popular_factors.alias("b")) \
    .filter(col("a.movie_id") < col("b.movie_id")) \
    .withColumn("similarity", cosine_sim(col("a.features"), col("b.features"))) \
    .select(
        col("a.movie_id").alias("movie_id_a"),
        col("b.movie_id").alias("movie_id_b"),
        "similarity"
    )

pairs.cache()
print(f"Par filmow: {pairs.count()}")
pairs.orderBy(desc("similarity")).show(10)

In [ ]:
# Top 10 podobnych filmow per film
from pyspark.sql.window import Window

SIM_K = 10

# Symetryczne pary (a->b i b->a)
all_pairs = pairs.union(
    pairs.select(
        col("movie_id_b").alias("movie_id_a"),
        col("movie_id_a").alias("movie_id_b"),
        "similarity"
    )
)

# Top K per film
w = Window.partitionBy("movie_id_a").orderBy(desc("similarity"))
top_similar = all_pairs.withColumn("rank", row_number().over(w)) \
    .filter(col("rank") <= SIM_K)

# Pivot do formatu plaskiego: movie_id, m_01, m_02, ..., m_10
sim_flat = top_similar.groupBy("movie_id_a").pivot("rank", list(range(1, SIM_K + 1))) \
    .agg(first("movie_id_b"))

# Rename kolumn
for i in range(1, SIM_K + 1):
    sim_flat = sim_flat.withColumnRenamed(str(i), f"m_{i:02d}")

sim_flat = sim_flat.withColumnRenamed("movie_id_a", "movie_id") \
    .withColumn("movie_key", lpad(col("movie_id").cast("string"), 6, "0"))

print(f"Tabela podobnych filmow: {sim_flat.count()} filmow")
sim_flat.show(5)

In [ ]:
# Zapisz tabele podobnych filmow do HBase
sim_columns = {
    "movie_key": {"cf": "rowkey", "col": "movie_key", "type": "string"},
    "movie_id":  {"cf": "info", "col": "movie_id", "type": "int"}
}
for i in range(1, SIM_K + 1):
    sim_columns[f"m_{i:02d}"] = {"cf": "similar", "col": f"m_{i:02d}", "type": "int"}

sim_catalog = json.dumps({
    "table": {"namespace": "default", "name": "movie_similarity"},
    "rowkey": "movie_key",
    "columns": sim_columns
})

start = time.time()
sim_flat.write \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=sim_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .option("newTable", "5") \
    .mode("overwrite") \
    .save()
sim_time = time.time() - start
print(f"Tabela movie_similarity zapisana w {sim_time:.1f}s")

In [ ]:
# Weryfikacja - filmy podobne do Toy Story (movie_id=1)
sim_from_hbase = spark.read \
    .format("org.apache.hadoop.hbase.spark") \
    .options(catalog=sim_catalog) \
    .option("hbase.spark.use.hbasecontext", "false") \
    .load()

toy_story_sim = sim_from_hbase.filter(col("movie_key") == "000001")
if toy_story_sim.count() > 0:
    ts = toy_story_sim.collect()[0]
    print("Filmy podobne do Toy Story (1995):")
    for i in range(1, SIM_K + 1):
        mid = ts[f"m_{i:02d}"]
        if mid:
            title = movies.filter(col("movie_id") == mid).select("title").collect()
            title_str = title[0]["title"] if title else "unknown"
            print(f"  {i:2d}. movie_id={mid} - {title_str}")
else:
    print("Toy Story nie jest w top 1000 popularnych filmow lub HBase niedostepny")

## 11. Spark reads z HBase - real-time feature enrichment

W produkcji Spark moze czytac z HBase w celu wzbogacenia danych (feature enrichment) - np. dolaczenie profilu uzytkownika do streamu zdarzen.

In [ ]:
# Symulacja: strumien nowych ocen + enrichment z HBase

# "Nowe" oceny (symulacja - ostatnie 1000 ocen)
new_ratings = ratings.orderBy(desc("rating_timestamp")).limit(1000)

# Dolacz rekomendacje z HBase - czy user ocenil film ktory mu polecilismy?
enriched = new_ratings.withColumn(
    "user_key", lpad(col("user_id").cast("string"), 6, "0")
).join(
    recs_from_hbase.select("user_key", "r_01", "r_02", "r_03", "r_04", "r_05"),
    "user_key",
    "left"
)

# Sprawdz czy oceniony film byl w top 5 rekomendacji
enriched = enriched.withColumn(
    "was_recommended",
    col("movie_id").isin(
        col("r_01"), col("r_02"), col("r_03"), col("r_04"), col("r_05")
    )
)

hit_rate = enriched.filter(col("was_recommended") == True).count() / enriched.count() * 100
print(f"Hit rate (film w top 5 recs): {hit_rate:.2f}%")

enriched.filter(col("was_recommended") == True) \
    .select("user_id", "movie_id", "rating", "r_01", "r_02", "r_03") \
    .show(10)

## Zadanie 1

Zaimplementuj sprawdzanie swiezosci rekomendacji uzywajac timestampow.

1. Zapisz rekomendacje do HBase z kolumna `meta:generated_at` (timestamp generowania)
2. Odczytaj rekomendacje z HBase
3. Dla kazdego uzytkownika sprawdz czy rekomendacje sa "swieze" (< 7 dni)
4. Oblicz statystyki:
   - Ile procent uzytkownikow ma swieze rekomendacje?
   - Ile procent ma przestarzale rekomendacje (> 7 dni)?
   - Ile procent nie ma rekomendacji wcale?
5. Dla uzytkownikow z przestarzalymi rekomendacjami - wygeneruj nowe i zapisz do HBase

Wskazowka: Symuluj rozne daty generowania, np. losowo 1-14 dni wstecz.

In [ ]:
# Twoje rozwiazanie:


## Zadanie koncowe

Zbuduj pelny pipeline: train ALS, export do HBase, symulacja API reads.

### Kroki:

1. **Trening modelu ALS** na pelnych danych MovieLens
   - Uzyj optymalnych parametrow (rank=20, regParam=0.1, maxIter=15)
   - Zmierz RMSE na zbiorze testowym

2. **Generowanie i eksport rekomendacji:**
   - Top 20 rekomendacji per uzytkownik → tabela `pipeline_user_recs`
   - Top 10 podobnych filmow per film (top 500) → tabela `pipeline_movie_sim`
   - Profil uzytkownika (avg_rating, num_ratings, top_genre) → tabela `pipeline_user_profiles`
   - Uzyj bulk load dla duzych tabel

3. **Symulacja API reads** (GET dla 100 losowych uzytkownikow):
   - Pobierz rekomendacje z HBase
   - Dolacz tytuly filmow
   - Zmierz sredni czas odpowiedzi
   - Porownaj z PostgreSQL (Spark JDBC)

4. **Raport:**
   - Czas treningu modelu
   - Czas generowania rekomendacji
   - Czas zapisu do HBase (puts vs bulk load)
   - Sredni czas odczytu z HBase vs PostgreSQL
   - RMSE modelu
   - Przykladowe rekomendacje dla 3 uzytkownikow

In [ ]:
# Twoje rozwiazanie:


In [ ]:
ratings.unpersist()
movies.unpersist()
training.unpersist()
all_recs.unpersist()
pairs.unpersist()
spark.stop()